# <left> Benchmark Serialization </left> 

In [1]:
import time
import os

import numpy as np
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import pickle
import joblib

import onnxruntime
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from sklearn2pmml import PMMLPipeline
from sklearn2pmml import sklearn2pmml
from pypmml import Model

from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')

# Определение путей для сохранения моделей
models_dir = "models"
models_path = os.path.join("..", models_dir)


In [2]:
def create_models_dir():
    """Создание директории для моделей, если она не существует"""
    os.makedirs(models_path, exist_ok=True)
    print(f"\n📁 Директория для моделей создана: {models_path}")

def generate_data(n_samples=10000, n_features=20):
    """Генерация синтетических данных"""
    print("\n🔧 Генерация синтетических данных...")
    X, y = make_classification(
        n_samples=n_samples,
        n_features=n_features,
        n_informative=15,
        n_redundant=5,
        random_state=42
    )
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    print(f"✅ Данные сгенерированы: {n_samples} сэмплов, {n_features} признаков")
    return X_train, X_test, y_train, y_test

def create_pipeline():
    """Создание пайплайна"""
    return Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=4)),
        ('clf', RandomForestClassifier(n_estimators=100, random_state=42))
    ])

def train_model(X_train, y_train, X_test, y_test):
    """Обучение модели"""
    print("\n🚀 Обучение пайплайна...")
    model = create_pipeline()
    model.fit(X_train, y_train)
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)
    print(f"✅ Пайплайн обучен: Train accuracy = {train_score:.4f}, Test accuracy = {test_score:.4f}")
    return model

def save_models(model, X_train, y_train):
    """Сохранение модели в различных форматах"""
    print("\n💾 Сохранение пайплайна в различных форматах...")
    create_models_dir()
    save_paths = {}
    
    # Pickle
    pickle_path = os.path.join(models_path, 'model.pkl')
    with open(pickle_path, 'wb') as f:
        pickle.dump(model, f)
    save_paths['pickle'] = pickle_path

    # Joblib
    joblib_path = os.path.join(models_path, 'model.joblib')
    joblib.dump(model, joblib_path)
    save_paths['joblib'] = joblib_path

    # ONNX
    initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]
    onnx_model = convert_sklearn(model, initial_types=initial_type)
    onnx_path = os.path.join(models_path, 'model.onnx')
    with open(onnx_path, "wb") as f:
        f.write(onnx_model.SerializeToString())
    save_paths['onnx'] = onnx_path

    # PMML
    pmml_pipeline = PMMLPipeline([("pipeline", model)])
    pmml_pipeline.fit(X_train, y_train)
    pmml_path = os.path.join(models_path, 'model.pmml')
    sklearn2pmml(pmml_pipeline, pmml_path)
    save_paths['pmml'] = pmml_path

    print("✅ Пайплайны сохранены во всех форматах")
    return save_paths

def benchmark_models(save_paths, X_test):
    """Измерение производительности моделей"""
    print("\n📊 Начало бенчмаркинга...")
    results = []

    for format_name, path in save_paths.items():
        # Измерение размера файла
        size_mb = os.path.getsize(path) / (1024 * 1024)
        
        # Загрузка модели и измерение времени инференса
        if format_name == 'pickle':
            with open(path, 'rb') as f:
                model = pickle.load(f)
            start_time = time.time()
            predictions = model.predict(X_test)
            inference_time = (time.time() - start_time) * 1000

        elif format_name == 'joblib':
            model = joblib.load(path)
            start_time = time.time()
            predictions = model.predict(X_test)
            inference_time = (time.time() - start_time) * 1000

        elif format_name == 'onnx':
            session = onnxruntime.InferenceSession(path)
            input_name = session.get_inputs()[0].name
            start_time = time.time()
            predictions = session.run(None, {input_name: X_test.astype(np.float32)})[0]
            inference_time = (time.time() - start_time) * 1000

        elif format_name == 'pmml':
            model = Model.load(path)
            start_time = time.time()
            predictions = model.predict(X_test)
            inference_time = (time.time() - start_time) * 1000
        
        results.append([
            format_name,
            f"{size_mb:.2f}",
            f"{inference_time:.2f}"
        ])

    # Вывод результатов в виде таблицы
    headers = ["Формат", "Размер (MB)", "Время инференса (ms)"]
    print("\n📈 Результаты бенчмарка:")
    print(tabulate(results, headers=headers, tablefmt="grid"))
    return results

def cleanup_files(save_paths):
    """Очистка временных файлов"""
    for path in save_paths.values():
        if os.path.exists(path):
            os.remove(path)
    print("\n🧹 Временные файлы удалены")

In [3]:
# Генерация данных
X_train, X_test, y_train, y_test = generate_data()

# Обучение модели
model = train_model(X_train, y_train, X_test, y_test)

# Сохранение в разных форматах
save_paths = save_models(model, X_train, y_train)

# Проведение бенчмарка
results = benchmark_models(save_paths, X_test)

# Очистка файлов (опционально)
cleanup_files(save_paths)


🔧 Генерация синтетических данных...
✅ Данные сгенерированы: 10000 сэмплов, 20 признаков

🚀 Обучение пайплайна...
✅ Пайплайн обучен: Train accuracy = 0.9999, Test accuracy = 0.7600

💾 Сохранение пайплайна в различных форматах...

📁 Директория для моделей создана: ../models
✅ Пайплайны сохранены во всех форматах

📊 Начало бенчмаркинга...

📈 Результаты бенчмарка:
+----------+---------------+------------------------+
| Формат   |   Размер (MB) |   Время инференса (ms) |
+==========+===============+========================+
| pickle   |         18.61 |                  30.52 |
+----------+---------------+------------------------+
| joblib   |         18.62 |                  32.22 |
+----------+---------------+------------------------+
| onnx     |          9.03 |                  11.72 |
+----------+---------------+------------------------+
| pmml     |         48.94 |                1112.12 |
+----------+---------------+------------------------+

🧹 Временные файлы удалены
